In [1]:
pip install wandb -q

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:42433")

# Import

In [3]:
import numpy as np
import xarray as xr
import torch
import pytorch_lightning as pl
from scipy import ndimage
import itertools
import os
from pytorch_lightning.loggers import WandbLogger
import wandb
from pytorch_lightning.callbacks import EarlyStopping,ModelCheckpoint,Callback
import myParam

In [4]:
import platform
print(platform.platform())

Linux-5.10.133+-x86_64-with-glibc2.35


In [5]:
print(torch.__version__)

1.13.1.post200


In [6]:
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", ".*Consider increasing the value of the `num_workers` argument*")

In [7]:
import importlib

In [8]:
importlib.reload(myParam)

<module 'myParam' from '/home/jovyan/oceanDataNotebooks/parametrization_NN/myParam.py'>

## Open data

In [9]:
PERSISTENT_BUCKET = os.environ['PERSISTENT_BUCKET'] 

In [10]:
data_dict=[dict() for i in range(6)]
data_dict[0] = dict({'region' : '1', 'season' : 'fma', 'label' : 'GULFSTR FMA'})
data_dict[1] = dict({'region' : '1', 'season' : 'aso', 'label' : 'GULFSTR ASO'})
data_dict[2] = dict({'region' : '2', 'season' : 'fma', 'label' : 'MIDATL FMA'})
data_dict[3] = dict({'region' : '2', 'season' : 'aso', 'label' : 'MIDATL ASO'})
data_dict[4] = dict({'region' : '3', 'season' : 'fma', 'label' : 'WESTMED FMA'})
data_dict[5] = dict({'region' : '3', 'season' : 'aso', 'label' : 'WESTMED ASO'})

In [11]:
batch_size = 4
height = 45
width = 40

In [12]:
%%time
features_to_add_to_sample = ['votemper', 'votemper_var', 'rho_ct_ct', 'diff_votemper_sqr']
auxiliary_features = ['z_l', 'f', 'e1t', 'e2t']
all_data_3D = myParam.PyLiDataModule(data_dict, '3D', features_to_add_to_sample, auxiliary_features, height, width, batch_size=batch_size)

CPU times: user 17.8 s, sys: 3.46 s, total: 21.2 s
Wall time: 1min 23s


In [13]:
%%time
features_to_add_to_sample = ['sosstsst', 'sosstsst_var', 'rho_ct_ct', 'diff_sosstsst_sqr']
auxiliary_features = ['e1t', 'e2t']
all_data_2D = myParam.PyLiDataModule(data_dict, '2D', features_to_add_to_sample, auxiliary_features, height, width, batch_size=batch_size)

CPU times: user 1.98 s, sys: 481 ms, total: 2.46 s
Wall time: 12.6 s


# Run experiment routine

In [14]:
def run_experiment(config, project) :
    wandb_logger = WandbLogger(name=config['model_label']+'_'+config['version'], \
                               version=config['model_label']+'_'+config['version'],\
                               project=project, config=config, resume=False, log_model=True, offline=False)
    
    torch_model = eval(config['torch_model'])(**config['torch_model_params'])
    pylight_module = myParam.GenericPyLiModule(torch_model, **config['module_params'])
    
    # Callbacks
    checkpoint_callback = ModelCheckpoint(monitor="loss_train", save_last=True)    
    early_stopping_callback = EarlyStopping(monitor="loss_train", mode="min")
    
    trainer = pl.Trainer(**config['training_params'], logger=wandb_logger, 
                     callbacks=[early_stopping_callback, checkpoint_callback],
                     accelerator='gpu', devices=(1 if torch.cuda.is_available() else None))  
    trainer.fit(model = pylight_module, datamodule=eval(config['datamodule']))
    #tests
    test_datamodule = eval(config['datamodule'])
    test_datamodule.setup(stage='test')
    trainer.test(model = pylight_module, datamodule=test_datamodule)
    wandb.finish()

# Runs

In [15]:
wandb.finish()

In [16]:
project_name = 'tests'

## Linear regression

In [17]:
config=dict({'model_label' : 'LinReg',
                'version' : 'HuberLoss2D',
                'torch_model' : 'myParam.lin_regr_model',
                'datamodule' : 'all_data_2D',
                'torch_model_params' : dict({'data_geometry' : '2D',\
                                            'nb_of_input_features' : 1, \
                                            'nb_of_output_features' : 1}),
                'module_params' : dict({'input_features'  : ['diff_sosstsst_sqr'],
                                        'output_features'  : ['sosstsst_var'],
                                        'output_units' : None,
                                        'loss' : torch.nn.functional.huber_loss,
                                        'optimizer' : torch.optim.SGD,
                                        'learning_rate' : 1e-3,}),
                'training_params' : dict({'max_epochs' : 100,
                                          'limit_train_batches' : 1.0})
               })
run_experiment(config, project_name)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: anagorb63. Use `wandb login --relogin` to force relogin


/srv/conda/envs/notebook/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'torch_model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['torch_model'])`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
/srv/conda/envs/notebook/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:604: UserWarning: Checkpoint directory ./tests/LinReg_HuberLoss2D/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type           | Params
-----------------------------------------------
0 | torch_model |

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃        DataLoader 0        ┃       DataLoader 1        ┃        DataLoader 2        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ corr_coef_grad_sosstsst_… │     0.7017475366592407     │    0.7641322612762451     │     0.5873867869377136     │
│  corr_coef_sosstsst_var   │     0.7137863636016846     │    0.7573875784873962     │     0.7858012914657593     │
│  loss_grad_sosstsst_var   │     22.566877365112305     │    1.2401963472366333     │     1.2416118383407593     │
│   loss_val_sosstsst_var   │     0.5894184708595276     │    0.11327831447124481    │    0.048786409199237823    │
└───────────────────────────┴────────────────────────────┴───────────────────────────┴────────────────────────────┘

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃        DataLoader 3        ┃       DataLoader 4        ┃        DataLoader 5        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ corr_coef_grad_sosstsst_… │     0.6714096665382385     │    0.2380993664264679     │     0.525032103061676      │
│  corr_coef_sosstsst_var   │     0.8019290566444397     │    0.49192318320274353    │     0.5965465307235718     │
│  loss_grad_sosstsst_var   │    0.03594658896327019     │  1.4818753015788388e-06   │   2.458786366332788e-05    │
│   loss_val_sosstsst_var   │    0.00855300948023796     │  0.00016202623373828828   │    0.000961391138844192    │
└───────────────────────────┴────────────────────────────┴───────────────────────────┴────────────────────────────┘

corr_coef_grad_sosstsst_var/dataloader_idx_0,▁
corr_coef_grad_sosstsst_var/dataloader_idx_1,▁
corr_coef_grad_sosstsst_var/dataloader_idx_2,▁
corr_coef_grad_sosstsst_var/dataloader_idx_3,▁
corr_coef_grad_sosstsst_var/dataloader_idx_4,▁
corr_coef_grad_sosstsst_var/dataloader_idx_5,▁
corr_coef_sosstsst_var/dataloader_idx_0,▁
corr_coef_sosstsst_var/dataloader_idx_1,▁
corr_coef_sosstsst_var/dataloader_idx_2,▁
corr_coef_sosstsst_var/dataloader_idx_3,▁
corr_coef_sosstsst_var/dataloader_idx_4,▁


In [18]:
config=dict({'model_label' : 'LinReg',
                'version' : 'HuberLoss3D',
                'torch_model' : 'myParam.lin_regr_model',
                'datamodule' : 'all_data_3D',
                'torch_model_params' : dict({'data_geometry' : '3D',\
                                            'nb_of_input_features' : 1, \
                                            'nb_of_output_features' : 1}),
                'module_params' : dict({'input_features'  : ['diff_votemper_sqr'],
                                        'output_features'  : ['votemper_var'],
                                        'output_units' : None,
                                        'loss' : torch.nn.functional.huber_loss,
                                        'optimizer' : torch.optim.SGD,
                                        'learning_rate' : 1e-3,}),
                'training_params' : dict({'max_epochs' : 100,
                                          'limit_train_batches' : 1.0})
               })
run_experiment(config, project_name)

/srv/conda/envs/notebook/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'torch_model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['torch_model'])`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
/srv/conda/envs/notebook/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:604: UserWarning: Checkpoint directory ./tests/LinReg_HuberLoss3D/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type           | Params
-----------------------------------------------
0 | torch_model |

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃        DataLoader 0        ┃       DataLoader 1        ┃        DataLoader 2        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ corr_coef_grad_votemper_… │     0.6990418434143066     │    0.6940702795982361     │     0.6482260227203369     │
│  corr_coef_votemper_var   │     0.7958126664161682     │    0.7883097529411316     │     0.8395946621894836     │
│  loss_grad_votemper_var   │     1.1463968753814697     │     0.524319589138031     │     0.3721678555011749     │
│  loss_val_pressure_grad   │   1.131532676567395e-08    │  1.2215566271819545e-08   │   1.2847153504039265e-08   │
│ loss_val_pressure_grad_x  │   4.620095751535802e-09    │   5.423403217702738e-09   │   6.9798197235695395e-09   │
│ loss_val_pressure_grad_y  │   6.695231014138148e-09    │   6.792163054116806e-09   │   5.867333780469726e-09    │
│   loss_val_votemper_var   │    0.07995761930942535     │   0.046734586358070374    │    0.01815192960202694     │
└───────────────────────────┴────────────────────────────┴───────────────────────────┴────────────────────────────┘

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃        DataLoader 3        ┃       DataLoader 4        ┃        DataLoader 5        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ corr_coef_grad_votemper_… │     0.7831739187240601     │    0.5086860060691833     │     0.5103292465209961     │
│  corr_coef_votemper_var   │     0.857978105545044      │    0.7896470427513123     │     0.7639338374137878     │
│  loss_grad_votemper_var   │    0.36409473419189453     │  2.3510430935402837e-07   │   0.00014089093019720167   │
│  loss_val_pressure_grad   │   6.268158523727805e-09    │   8.639746168194157e-12   │   5.4026837169430826e-11   │
│ loss_val_pressure_grad_x  │   3.079886542018138e-09    │   4.060544594440221e-12   │   2.633084947061313e-11    │
│ loss_val_pressure_grad_y  │   3.1882719817096673e-09   │  4.5792015737539354e-12   │   2.7695987698817706e-11   │
│   loss_val_votemper_var   │    0.012032397091388702    │   6.161496276035905e-05   │   0.0008162074955180287    │
└───────────────────────────┴────────────────────────────┴───────────────────────────┴────────────────────────────┘

corr_coef_grad_votemper_var/dataloader_idx_0,▁
corr_coef_grad_votemper_var/dataloader_idx_1,▁
corr_coef_grad_votemper_var/dataloader_idx_2,▁
corr_coef_grad_votemper_var/dataloader_idx_3,▁
corr_coef_grad_votemper_var/dataloader_idx_4,▁
corr_coef_grad_votemper_var/dataloader_idx_5,▁
corr_coef_votemper_var/dataloader_idx_0,▁
corr_coef_votemper_var/dataloader_idx_1,▁
corr_coef_votemper_var/dataloader_idx_2,▁
corr_coef_votemper_var/dataloader_idx_3,▁
corr_coef_votemper_var/dataloader_idx_4,▁


In [19]:
config=dict({'model_label' : 'LinReg',
                'version' : 'MSELoss2D',
                'torch_model' : 'myParam.lin_regr_model',
                'datamodule' : 'all_data_2D',
                'torch_model_params' : dict({'data_geometry' : '2D',\
                                            'nb_of_input_features' : 1, \
                                            'nb_of_output_features' : 1}),
                'module_params' : dict({'input_features'  : ['diff_sosstsst_sqr'],
                                        'output_features'  : ['sosstsst_var'],
                                        'output_units' : None,
                                        'loss' : torch.nn.functional.mse_loss,
                                        'optimizer' : torch.optim.SGD,
                                        'learning_rate' : 1e-3,}),
                'training_params' : dict({'max_epochs' : 100,
                                          'limit_train_batches' : 1.0})
               })
run_experiment(config, project_name)

/srv/conda/envs/notebook/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'torch_model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['torch_model'])`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
/srv/conda/envs/notebook/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:604: UserWarning: Checkpoint directory ./tests/LinReg_MSELoss2D/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type           | Params
-----------------------------------------------
0 | torch_model | l

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃        DataLoader 0        ┃       DataLoader 1        ┃        DataLoader 2        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ corr_coef_grad_sosstsst_… │     0.701748251914978      │    0.7641282677650452     │     0.587380588054657      │
│  corr_coef_sosstsst_var   │     0.7137874960899353     │    0.7573866844177246     │     0.7858011722564697     │
│  loss_grad_sosstsst_var   │     18.930721282958984     │    0.9677502512931824     │     1.185933232307434      │
│   loss_val_sosstsst_var   │     0.5531823635101318     │    0.10663110762834549    │    0.05596914142370224     │
└───────────────────────────┴────────────────────────────┴───────────────────────────┴────────────────────────────┘

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃        DataLoader 3        ┃       DataLoader 4        ┃        DataLoader 5        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ corr_coef_grad_sosstsst_… │     0.6714114546775818     │    0.23809805512428284    │     0.5250321626663208     │
│  corr_coef_sosstsst_var   │     0.8019294142723083     │    0.49191802740097046    │     0.5965473055839539     │
│  loss_grad_sosstsst_var   │    0.031341638416051865    │   1.260864905816561e-06   │   2.1941195882391185e-05   │
│   loss_val_sosstsst_var   │    0.01093300525099039     │   0.0001534304319648072   │   0.0009503160836175084    │
└───────────────────────────┴────────────────────────────┴───────────────────────────┴────────────────────────────┘

corr_coef_grad_sosstsst_var/dataloader_idx_0,▁
corr_coef_grad_sosstsst_var/dataloader_idx_1,▁
corr_coef_grad_sosstsst_var/dataloader_idx_2,▁
corr_coef_grad_sosstsst_var/dataloader_idx_3,▁
corr_coef_grad_sosstsst_var/dataloader_idx_4,▁
corr_coef_grad_sosstsst_var/dataloader_idx_5,▁
corr_coef_sosstsst_var/dataloader_idx_0,▁
corr_coef_sosstsst_var/dataloader_idx_1,▁
corr_coef_sosstsst_var/dataloader_idx_2,▁
corr_coef_sosstsst_var/dataloader_idx_3,▁
corr_coef_sosstsst_var/dataloader_idx_4,▁


In [20]:
config=dict({'model_label' : 'LinReg',
                'version' : 'MSELoss3D',
                'torch_model' : 'myParam.lin_regr_model',
                'datamodule' : 'all_data_3D',
                'torch_model_params' : dict({'data_geometry' : '3D',\
                                            'nb_of_input_features' : 1, \
                                            'nb_of_output_features' : 1}),
                'module_params' : dict({'input_features'  : ['diff_votemper_sqr'],
                                        'output_features'  : ['votemper_var'],
                                        'output_units' : None,
                                        'loss' : torch.nn.functional.mse_loss,
                                        'optimizer' : torch.optim.SGD,
                                        'learning_rate' : 1e-3,}),
                'training_params' : dict({'max_epochs' : 100,
                                          'limit_train_batches' : 1.0})
               })
run_experiment(config, project_name)

/srv/conda/envs/notebook/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'torch_model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['torch_model'])`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
/srv/conda/envs/notebook/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:604: UserWarning: Checkpoint directory ./tests/LinReg_MSELoss3D/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type           | Params
-----------------------------------------------
0 | torch_model | l

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃        DataLoader 0        ┃       DataLoader 1        ┃        DataLoader 2        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ corr_coef_grad_votemper_… │     0.6990439891815186     │    0.6940703392028809     │     0.6482446789741516     │
│  corr_coef_votemper_var   │     0.7958106398582458     │     0.788307785987854     │     0.8395931124687195     │
│  loss_grad_votemper_var   │     1.010498285293579      │    0.4460529088973999     │     0.3404309153556824     │
│  loss_val_pressure_grad   │   1.0524437062796312e-08   │   1.000193218099007e-08   │   1.277788078683281e-08    │
│ loss_val_pressure_grad_x  │   4.321333148244329e-09    │   4.57481468623353e-09    │   6.847951751491284e-09    │
│ loss_val_pressure_grad_y  │   6.203103914551984e-09    │   5.427117494756541e-09   │   5.929929035341526e-09    │
│   loss_val_votemper_var   │    0.07706894725561142     │   0.045105911791324615    │    0.01743333600461483     │
└───────────────────────────┴────────────────────────────┴───────────────────────────┴────────────────────────────┘

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃        DataLoader 3        ┃       DataLoader 4        ┃        DataLoader 5        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ corr_coef_grad_votemper_… │     0.7831082344055176     │    0.5086991786956787     │     0.510320782661438      │
│  corr_coef_votemper_var   │     0.8579773902893066     │    0.7896402478218079     │     0.7639501094818115     │
│  loss_grad_votemper_var   │    0.30711647868156433     │  2.0970760772343056e-07   │   0.00012198056356282905   │
│  loss_val_pressure_grad   │   5.797247171440368e-09    │   7.997597387121102e-12   │   5.1177471732213666e-11   │
│ loss_val_pressure_grad_x  │   2.878587289345786e-09    │   3.824321950558324e-12   │   2.4924629651314188e-11   │
│ loss_val_pressure_grad_y  │   2.9186598820945824e-09   │   4.173275436562778e-12   │   2.6252842080899475e-11   │
│   loss_val_votemper_var   │    0.012013738043606281    │   5.933816282777116e-05   │   0.0007790336385369301    │
└───────────────────────────┴────────────────────────────┴───────────────────────────┴────────────────────────────┘

corr_coef_grad_votemper_var/dataloader_idx_0,▁
corr_coef_grad_votemper_var/dataloader_idx_1,▁
corr_coef_grad_votemper_var/dataloader_idx_2,▁
corr_coef_grad_votemper_var/dataloader_idx_3,▁
corr_coef_grad_votemper_var/dataloader_idx_4,▁
corr_coef_grad_votemper_var/dataloader_idx_5,▁
corr_coef_votemper_var/dataloader_idx_0,▁
corr_coef_votemper_var/dataloader_idx_1,▁
corr_coef_votemper_var/dataloader_idx_2,▁
corr_coef_votemper_var/dataloader_idx_3,▁
corr_coef_votemper_var/dataloader_idx_4,▁


## FCNN

In [21]:
config = dict({'model_label' : 'FCNN',
                'version' : 'patch_in3_out1_val_loss3D_nn_nf',
                'torch_model' : 'myParam.FCNN',
                'datamodule' : 'all_data_3D',
                'torch_model_params' : dict({'data_geometry' : '3D',\
                                            'nb_of_input_features' : 1, \
                                            'nb_of_output_features' : 1, \
                                            'input_patch_size' : 3,
                                            'output_patch_size' : 1, 
                                            'int_layer_width' : 50}),
                'module_params' : dict({'input_features'  : ['votemper'],
                                        'output_features'  : ['votemper_var'],
                                        'output_units' : ['diff_votemper_sqr'],
                                        'loss' : torch.nn.functional.mse_loss,
                                        'optimizer' : torch.optim.Adam,
                                        'learning_rate' : 1e-3,}),
                'training_params' : dict({'max_epochs' : 100,
                                          'limit_train_batches' : 1.0})
               })
run_experiment(config, project_name)

/srv/conda/envs/notebook/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'torch_model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['torch_model'])`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type | Params
-------------------------------------
0 | torch_model | FCNN | 3.1 K 
-------------------------------------
3.1 K     Trainable params
0         Non-trainable params
3.1 K     Total params
0.012     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃        DataLoader 0        ┃       DataLoader 1        ┃        DataLoader 2        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ corr_coef_grad_votemper_… │     0.6576245427131653     │    0.5826008319854736     │     0.5378803610801697     │
│  corr_coef_votemper_var   │     0.8254888653755188     │    0.8173369765281677     │     0.7789085507392883     │
│  loss_grad_votemper_var   │     0.801144540309906      │    0.4878673553466797     │    0.39402973651885986     │
│  loss_val_pressure_grad   │   1.6250481751568312e-08   │  1.6060057959420397e-08   │   2.4736964745741803e-08   │
│ loss_val_pressure_grad_x  │   8.584784071453151e-09    │   9.536615473800787e-09   │   1.2809833305476424e-08   │
│ loss_val_pressure_grad_y  │   7.665697680115162e-09    │  6.5234424856196085e-09   │   1.1927131440265377e-08   │
│   loss_val_votemper_var   │    0.05621855705976486     │   0.024662019684910774    │    0.025645434856414795    │
└───────────────────────────┴────────────────────────────┴───────────────────────────┴────────────────────────────┘

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃        DataLoader 3        ┃       DataLoader 4        ┃        DataLoader 5        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ corr_coef_grad_votemper_… │     0.6239852905273438     │    0.5324996709823608     │     0.4917779266834259     │
│  corr_coef_votemper_var   │     0.8068744540214539     │    0.7824996709823608     │     0.7471328973770142     │
│  loss_grad_votemper_var   │     0.4394199848175049     │   2.675211874247907e-07   │   0.00012246696860529482   │
│  loss_val_pressure_grad   │   1.1871614799045005e-08   │  1.0631012781870262e-11   │   5.686801982540598e-11    │
│ loss_val_pressure_grad_x  │   6.482037006688402e-09    │   4.816222425610848e-12   │   2.7305782717624966e-11   │
│ loss_val_pressure_grad_y  │   5.389577792356603e-09    │  5.8147903562594145e-12   │   2.9562237107781024e-11   │
│   loss_val_votemper_var   │    0.020174290984869003    │   7.080581417540088e-05   │   0.0008680668543092906    │
└───────────────────────────┴────────────────────────────┴───────────────────────────┴────────────────────────────┘

corr_coef_grad_votemper_var/dataloader_idx_0,▁
corr_coef_grad_votemper_var/dataloader_idx_1,▁
corr_coef_grad_votemper_var/dataloader_idx_2,▁
corr_coef_grad_votemper_var/dataloader_idx_3,▁
corr_coef_grad_votemper_var/dataloader_idx_4,▁
corr_coef_grad_votemper_var/dataloader_idx_5,▁
corr_coef_votemper_var/dataloader_idx_0,▁
corr_coef_votemper_var/dataloader_idx_1,▁
corr_coef_votemper_var/dataloader_idx_2,▁
corr_coef_votemper_var/dataloader_idx_3,▁
corr_coef_votemper_var/dataloader_idx_4,▁


In [22]:
config = dict({'model_label' : 'FCNN',
                'version' : 'patch_in3_out1_val_loss2D_nn_nf',
                'torch_model' : 'myParam.FCNN',
                'datamodule' : 'all_data_2D',
                'torch_model_params' : dict({'data_geometry' : '2D',\
                                            'nb_of_input_features' : 1, \
                                            'nb_of_output_features' : 1, \
                                            'input_patch_size' : 3,
                                            'output_patch_size' : 1, 
                                            'int_layer_width' : 50}),
                'module_params' : dict({'input_features'  : ['sosstsst'],
                                        'output_features'  : ['sosstsst_var'],
                                        'output_units' : ['diff_sosstsst_sqr'],
                                        'loss' : torch.nn.functional.mse_loss,
                                        'optimizer' : torch.optim.Adam,
                                        'learning_rate' : 1e-3,}),
                'training_params' : dict({'max_epochs' : 100,
                                          'limit_train_batches' : 1.0})
               })
run_experiment(config, project_name)

/srv/conda/envs/notebook/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'torch_model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['torch_model'])`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type | Params
-------------------------------------
0 | torch_model | FCNN | 3.1 K 
-------------------------------------
3.1 K     Trainable params
0         Non-trainable params
3.1 K     Total params
0.012     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃        DataLoader 0        ┃       DataLoader 1        ┃        DataLoader 2        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ corr_coef_grad_sosstsst_… │     0.8183562159538269     │    0.7653243541717529     │     0.5693227648735046     │
│  corr_coef_sosstsst_var   │     0.877980649471283      │    0.8652409911155701     │     0.7101638913154602     │
│  loss_grad_sosstsst_var   │     10.218339920043945     │    0.7178873419761658     │     2.2656075954437256     │
│   loss_val_sosstsst_var   │    0.23802074790000916     │   0.061657391488552094    │    0.09036979079246521     │
└───────────────────────────┴────────────────────────────┴───────────────────────────┴────────────────────────────┘

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃        DataLoader 3        ┃       DataLoader 4        ┃        DataLoader 5        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ corr_coef_grad_sosstsst_… │     0.7553076148033142     │    0.22320424020290375    │     0.5169235467910767     │
│  corr_coef_sosstsst_var   │     0.8420421481132507     │    0.5078906416893005     │     0.6241394281387329     │
│  loss_grad_sosstsst_var   │    0.03545181080698967     │  1.3071473858872196e-06   │   2.1119833036209457e-05   │
│   loss_val_sosstsst_var   │    0.009746531955897808    │  0.00014619520516134799   │   0.0008565891766920686    │
└───────────────────────────┴────────────────────────────┴───────────────────────────┴────────────────────────────┘

corr_coef_grad_sosstsst_var/dataloader_idx_0,▁
corr_coef_grad_sosstsst_var/dataloader_idx_1,▁
corr_coef_grad_sosstsst_var/dataloader_idx_2,▁
corr_coef_grad_sosstsst_var/dataloader_idx_3,▁
corr_coef_grad_sosstsst_var/dataloader_idx_4,▁
corr_coef_grad_sosstsst_var/dataloader_idx_5,▁
corr_coef_sosstsst_var/dataloader_idx_0,▁
corr_coef_sosstsst_var/dataloader_idx_1,▁
corr_coef_sosstsst_var/dataloader_idx_2,▁
corr_coef_sosstsst_var/dataloader_idx_3,▁
corr_coef_sosstsst_var/dataloader_idx_4,▁


In [23]:
config = dict({'model_label' : 'FCNN',
                'version' : 'patch_in3_out1_val_loss2D_nor_nf',
                'torch_model' : 'myParam.FCNN',
                'datamodule' : 'all_data_2D',
                'torch_model_params' : dict({'data_geometry' : '2D',\
                                            'nb_of_input_features' : 1, \
                                            'nb_of_output_features' : 1, \
                                            'input_patch_size' : 3,
                                            'output_patch_size' : 1, 
                                            'int_layer_width' : 50}),
                'module_params' : dict({'input_features'  : ['normalized_sosstsst'],
                                        'output_features'  : ['sosstsst_var'],
                                        'output_units' : ['diff_sosstsst_sqr'],
                                        'loss' : torch.nn.functional.mse_loss,
                                        'optimizer' : torch.optim.Adam,
                                        'learning_rate' : 1e-3,}),
                'training_params' : dict({'max_epochs' : 100,
                                          'limit_train_batches' : 1.0})
               })
run_experiment(config, project_name)

/srv/conda/envs/notebook/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'torch_model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['torch_model'])`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
/srv/conda/envs/notebook/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:604: UserWarning: Checkpoint directory ./tests/FCNN_patch_in3_out1_val_loss2D_nor_nf/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type | Params
-------------------------------------
0 | torch_model | 

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃        DataLoader 0        ┃       DataLoader 1        ┃        DataLoader 2        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ corr_coef_grad_sosstsst_… │     0.8478993773460388     │    0.8194363713264465     │     0.3646734356880188     │
│  corr_coef_sosstsst_var   │     0.8907948136329651     │    0.8891425728797913     │     0.2915504276752472     │
│  loss_grad_sosstsst_var   │     9.053519248962402      │    0.7390385270118713     │     465.18414306640625     │
│   loss_val_sosstsst_var   │    0.20655885338783264     │    0.05662650614976883    │     1.3530757427215576     │
└───────────────────────────┴────────────────────────────┴───────────────────────────┴────────────────────────────┘

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃        DataLoader 3        ┃       DataLoader 4        ┃        DataLoader 5        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ corr_coef_grad_sosstsst_… │     0.6888788342475891     │    0.1452183574438095     │    0.48660749197006226     │
│  corr_coef_sosstsst_var   │     0.7512838840484619     │    0.4922787547111511     │     0.5976707935333252     │
│  loss_grad_sosstsst_var   │    0.05634629353880882     │   1.100726512959227e-05   │   2.2843803890282288e-05   │
│   loss_val_sosstsst_var   │    0.01308119110763073     │  0.00013807787036057562   │   0.0008342464570887387    │
└───────────────────────────┴────────────────────────────┴───────────────────────────┴────────────────────────────┘

corr_coef_grad_sosstsst_var/dataloader_idx_0,▁
corr_coef_grad_sosstsst_var/dataloader_idx_1,▁
corr_coef_grad_sosstsst_var/dataloader_idx_2,▁
corr_coef_grad_sosstsst_var/dataloader_idx_3,▁
corr_coef_grad_sosstsst_var/dataloader_idx_4,▁
corr_coef_grad_sosstsst_var/dataloader_idx_5,▁
corr_coef_sosstsst_var/dataloader_idx_0,▁
corr_coef_sosstsst_var/dataloader_idx_1,▁
corr_coef_sosstsst_var/dataloader_idx_2,▁
corr_coef_sosstsst_var/dataloader_idx_3,▁
corr_coef_sosstsst_var/dataloader_idx_4,▁


In [24]:
config = dict({'model_label' : 'FCNN',
                'version' : 'patch_in3_out1_val_loss2D_nor_fil',
                'torch_model' : 'myParam.FCNN',
                'datamodule' : 'all_data_2D',
                'torch_model_params' : dict({'data_geometry' : '2D',\
                                            'nb_of_input_features' : 1, \
                                            'nb_of_output_features' : 1, \
                                            'input_patch_size' : 3,
                                            'output_patch_size' : 1, 
                                            'int_layer_width' : 50}),
                'module_params' : dict({'input_features'  : ['normalized_sosstsst'],
                                        'output_features'  : ['sosstsst_var'],
                                        'output_units' : ['filtered_diff_sosstsst_sqr'],
                                        'loss' : torch.nn.functional.mse_loss,
                                        'optimizer' : torch.optim.Adam,
                                        'learning_rate' : 1e-3,}),
                'training_params' : dict({'max_epochs' : 100,
                                          'limit_train_batches' : 1.0})
               })
run_experiment(config, project_name)

/srv/conda/envs/notebook/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'torch_model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['torch_model'])`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type | Params
-------------------------------------
0 | torch_model | FCNN | 3.1 K 
-------------------------------------
3.1 K     Trainable params
0         Non-trainable params
3.1 K     Total params
0.012     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃        DataLoader 0        ┃       DataLoader 1        ┃        DataLoader 2        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ corr_coef_grad_sosstsst_… │     0.8602010607719421     │    0.8339241743087769     │    0.41129690408706665     │
│  corr_coef_sosstsst_var   │     0.8879628777503967     │    0.8877348303794861     │     0.5405120849609375     │
│  loss_grad_sosstsst_var   │     9.325515747070312      │    0.5721104741096497     │     666.2357788085938      │
│   loss_val_sosstsst_var   │    0.22273992002010345     │   0.052493128925561905    │     1.3945846557617188     │
└───────────────────────────┴────────────────────────────┴───────────────────────────┴────────────────────────────┘

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃        DataLoader 3        ┃       DataLoader 4        ┃        DataLoader 5        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ corr_coef_grad_sosstsst_… │     0.6530531644821167     │    0.33927780389785767    │    0.34927377104759216     │
│  corr_coef_sosstsst_var   │     0.7893475294113159     │     0.480412095785141     │     0.486918568611145      │
│  loss_grad_sosstsst_var   │    0.11173020303249359     │  1.5195585092442343e-06   │   2.6844254534807988e-05   │
│   loss_val_sosstsst_var   │    0.018671875819563866    │  0.00017922632105182856   │   0.0012491047382354736    │
└───────────────────────────┴────────────────────────────┴───────────────────────────┴────────────────────────────┘

corr_coef_grad_sosstsst_var/dataloader_idx_0,▁
corr_coef_grad_sosstsst_var/dataloader_idx_1,▁
corr_coef_grad_sosstsst_var/dataloader_idx_2,▁
corr_coef_grad_sosstsst_var/dataloader_idx_3,▁
corr_coef_grad_sosstsst_var/dataloader_idx_4,▁
corr_coef_grad_sosstsst_var/dataloader_idx_5,▁
corr_coef_sosstsst_var/dataloader_idx_0,▁
corr_coef_sosstsst_var/dataloader_idx_1,▁
corr_coef_sosstsst_var/dataloader_idx_2,▁
corr_coef_sosstsst_var/dataloader_idx_3,▁
corr_coef_sosstsst_var/dataloader_idx_4,▁


In [25]:
config = dict({'model_label' : 'FCNN',
                'version' : 'patch_in3_out1_val_loss2D_nor_dim',
                'torch_model' : 'myParam.FCNN',
                'datamodule' : 'all_data_2D',
                'torch_model_params' : dict({'data_geometry' : '2D',\
                                            'nb_of_input_features' : 1, \
                                            'nb_of_output_features' : 1, \
                                            'input_patch_size' : 3,
                                            'output_patch_size' : 1, 
                                            'int_layer_width' : 50}),
                'module_params' : dict({'input_features'  : ['normalized_sosstsst'],
                                        'output_features'  : ['normalized_sosstsst_var'],
                                        'output_units' : None,
                                        'loss' : torch.nn.functional.mse_loss,
                                        'optimizer' : torch.optim.Adam,
                                        'learning_rate' : 1e-3,}),
                'training_params' : dict({'max_epochs' : 100,
                                          'limit_train_batches' : 1.0})
               })
run_experiment(config, project_name)

/srv/conda/envs/notebook/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'torch_model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['torch_model'])`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type | Params
-------------------------------------
0 | torch_model | FCNN | 3.1 K 
-------------------------------------
3.1 K     Trainable params
0         Non-trainable params
3.1 K     Total params
0.012     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

AttributeError: type object 'PyLiDataModule' has no attribute 'tensor_restore_norm'

In [ ]:
config = dict({'model_label' : 'FCNN',
                'version' : 'patch_in3_out1_val_loss2D_nn_dim',
                'torch_model' : 'myParam.FCNN',
                'datamodule' : 'all_data_2D',
                'torch_model_params' : dict({'data_geometry' : '2D',\
                                            'nb_of_input_features' : 1, \
                                            'nb_of_output_features' : 1, \
                                            'input_patch_size' : 3,
                                            'output_patch_size' : 1, 
                                            'int_layer_width' : 50}),
                'module_params' : dict({'input_features'  : ['sosstsst'],
                                        'output_features'  : ['sosstsst_var'],
                                        'output_units' : None,
                                        'loss' : torch.nn.functional.mse_loss,
                                        'optimizer' : torch.optim.Adam,
                                        'learning_rate' : 1e-3,}),
                'training_params' : dict({'max_epochs' : 100,
                                          'limit_train_batches' : 1.0})
               })
run_experiment(config, project_name)

/srv/conda/envs/notebook/lib/python3.10/site-packages/pytorch_lightning/loggers/wandb.py:396: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
/srv/conda/envs/notebook/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:604: UserWarning: Checkpoint directory ./tests/FCNN_patch_in3_out1_val_loss2D_nor_dim/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type | Params
-------------------------------------
0 | torch_model | FCN

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
config = dict({'model_label' : 'FCNN',
                'version' : 'patch_in3_out1_val_loss2D_nn_filt',
                'torch_model' : 'myParam.FCNN',
                'datamodule' : 'all_data_2D',
                'torch_model_params' : dict({'data_geometry' : '2D',\
                                            'nb_of_input_features' : 1, \
                                            'nb_of_output_features' : 1, \
                                            'input_patch_size' : 3,
                                            'output_patch_size' : 1, 
                                            'int_layer_width' : 50}),
                'module_params' : dict({'input_features'  : ['sosstsst'],
                                        'output_features'  : ['sosstsst_var'],
                                        'output_units' : ['filtered_diff_sosstsst_sqr'],
                                        'loss' : torch.nn.functional.mse_loss,
                                        'optimizer' : torch.optim.Adam,
                                        'learning_rate' : 1e-3,}),
                'training_params' : dict({'max_epochs' : 100,
                                          'limit_train_batches' : 1.0})
               })
run_experiment(config, project_name)

In [ ]:
config = dict({'model_label' : 'FCNN',
                'version' : 'patch_in3_out1_val_hubloss2D_norm_filt',
                'torch_model' : 'myParam.FCNN',
                'datamodule' : 'all_data_2D',
                'torch_model_params' : dict({'data_geometry' : '2D',\
                                            'nb_of_input_features' : 1, \
                                            'nb_of_output_features' : 1, \
                                            'input_patch_size' : 3,
                                            'output_patch_size' : 1, 
                                            'int_layer_width' : 50}),
                'module_params' : dict({'input_features'  : ['normalized_sosstsst'],
                                        'output_features'  : ['sosstsst_var'],
                                        'output_units' : ['filtered_diff_sosstsst_sqr'],
                                        'loss' : torch.nn.functional.huber_loss,
                                        'optimizer' : torch.optim.Adam,
                                        'learning_rate' : 1e-3,}),
                'training_params' : dict({'max_epochs' : 100,
                                          'limit_train_batches' : 1.0})
               })
run_experiment(config, project_name)

In [ ]:
config = dict({'model_label' : 'FCNN',
                'version' : 'patch_in3_out1_val_hubertloss2D_norm_nf',
                'torch_model' : 'myParam.FCNN',
                'datamodule' : 'all_data_2D',
                'torch_model_params' : dict({'data_geometry' : '2D',\
                                            'nb_of_input_features' : 1, \
                                            'nb_of_output_features' : 1, \
                                            'input_patch_size' : 3,
                                            'output_patch_size' : 1, 
                                            'int_layer_width' : 50}),
                'module_params' : dict({'input_features'  : ['normalized_sosstsst'],
                                        'output_features'  : ['sosstsst_var'],
                                        'output_units' : ['diff_sosstsst_sqr'],
                                        'loss' : torch.nn.functional.huber_loss,
                                        'optimizer' : torch.optim.Adam,
                                        'learning_rate' : 1e-3,}),
                'training_params' : dict({'max_epochs' : 100,
                                          'limit_train_batches' : 1.0})
               })
run_experiment(config, project_name)

In [ ]:
config = dict({'model_label' : 'FCNN',
                'version' : 'patch_in3_out1_val_loss3D_nn_dim',
                'torch_model' : 'myParam.FCNN',
                'datamodule' : 'all_data_3D',
                'torch_model_params' : dict({'data_geometry' : '3D',\
                                            'nb_of_input_features' : 1, \
                                            'nb_of_output_features' : 1, \
                                            'input_patch_size' : 3,
                                            'output_patch_size' : 1, 
                                            'int_layer_width' : 50}),
                'module_params' : dict({'input_features'  : ['votemper'],
                                        'output_features'  : ['votemper_var'],
                                        'output_units' : None,
                                        'loss' : torch.nn.functional.mse_loss,
                                        'optimizer' : torch.optim.Adam,
                                        'learning_rate' : 1e-3,}),
                'training_params' : dict({'max_epochs' : 100,
                                          'limit_train_batches' : 1.0})
               })
run_experiment(config, project_name)

In [ ]:
config = dict({'model_label' : 'FCNN',
                'version' : 'patch_in3_out1_val_loss3D_nn_fil',
                'torch_model' : 'myParam.FCNN',
                'datamodule' : 'all_data_3D',
                'torch_model_params' : dict({'data_geometry' : '3D',\
                                            'nb_of_input_features' : 1, \
                                            'nb_of_output_features' : 1, \
                                            'input_patch_size' : 3,
                                            'output_patch_size' : 1, 
                                            'int_layer_width' : 50}),
                'module_params' : dict({'input_features'  : ['votemper'],
                                        'output_features'  : ['votemper_var'],
                                        'output_units' : ['filtered_diff_votemper_sqr'],
                                        'loss' : torch.nn.functional.mse_loss,
                                        'optimizer' : torch.optim.Adam,
                                        'learning_rate' : 1e-3,}),
                'training_params' : dict({'max_epochs' : 100,
                                          'limit_train_batches' : 1.0})
               })
run_experiment(config, project_name)

In [ ]:
config = dict({'model_label' : 'FCNN',
                'version' : 'patch_in3_out1_val_loss3D_norm_fil',
                'torch_model' : 'myParam.FCNN',
                'datamodule' : 'all_data_3D',
                'torch_model_params' : dict({'data_geometry' : '3D',\
                                            'nb_of_input_features' : 1, \
                                            'nb_of_output_features' : 1, \
                                            'input_patch_size' : 3,
                                            'output_patch_size' : 1, 
                                            'int_layer_width' : 50}),
                'module_params' : dict({'input_features'  : ['normalized_votemper'],
                                        'output_features'  : ['votemper_var'],
                                        'output_units' : ['filtered_diff_votemper_sqr'],
                                        'loss' : torch.nn.functional.mse_loss,
                                        'optimizer' : torch.optim.Adam,
                                        'learning_rate' : 1e-3,}),
                'training_params' : dict({'max_epochs' : 100,
                                          'limit_train_batches' : 1.0})
               })
run_experiment(config, project_name)

In [ ]:
config = dict({'model_label' : 'FCNN',
                'version' : 'patch_in3_out1_val_loss3D_norm_nf',
                'torch_model' : 'myParam.FCNN',
                'datamodule' : 'all_data_3D',
                'torch_model_params' : dict({'data_geometry' : '3D',\
                                            'nb_of_input_features' : 1, \
                                            'nb_of_output_features' : 1, \
                                            'input_patch_size' : 3,
                                            'output_patch_size' : 1, 
                                            'int_layer_width' : 50}),
                'module_params' : dict({'input_features'  : ['normalized_votemper'],
                                        'output_features'  : ['votemper_var'],
                                        'output_units' : ['diff_votemper_sqr'],
                                        'loss' : torch.nn.functional.mse_loss,
                                        'optimizer' : torch.optim.Adam,
                                        'learning_rate' : 1e-3,}),
                'training_params' : dict({'max_epochs' : 100,
                                          'limit_train_batches' : 1.0})
               })
run_experiment(config, project_name)

In [ ]:
config = dict({'model_label' : 'FCNN',
                'version' : 'patch_in3_out1_val_loss3D_norm_dim',
                'torch_model' : 'myParam.FCNN',
                'datamodule' : 'all_data_3D',
                'torch_model_params' : dict({'data_geometry' : '3D',\
                                            'nb_of_input_features' : 1, \
                                            'nb_of_output_features' : 1, \
                                            'input_patch_size' : 3,
                                            'output_patch_size' : 1, 
                                            'int_layer_width' : 50}),
                'module_params' : dict({'input_features'  : ['normalized_votemper'],
                                        'output_features'  : ['votemper_var'],
                                        'output_units' : None,
                                        'loss' : torch.nn.functional.mse_loss,
                                        'optimizer' : torch.optim.Adam,
                                        'learning_rate' : 1e-3,}),
                'training_params' : dict({'max_epochs' : 100,
                                          'limit_train_batches' : 1.0})
               })
run_experiment(config, project_name)

In [ ]:
config = dict({'model_label' : 'FCNN',
                'version' : 'patch_in3_out1_pres_loss_3D_nn_nf',
                'torch_model' : 'myParam.FCNN',
                'datamodule' : 'all_data_3D',
                'torch_model_params' : dict({'data_geometry' : '3D',\
                                            'nb_of_input_features' : 1, \
                                            'nb_of_output_features' : 1, \
                                            'input_patch_size' : 3,
                                            'output_patch_size' : 1, 
                                            'int_layer_width' : 50}),
                'module_params' : dict({'input_features'  : ['votemper'],
                                        'output_features'  : ['votemper_var'],
                                        'output_units' : ['diff_votemper_sqr'],
                                        'loss' : 'pressure_based_MSEloss',
                                        'optimizer' : torch.optim.Adam,
                                        'learning_rate' : 1e-3,}),
                'training_params' : dict({'max_epochs' : 100,
                                          'limit_train_batches' : 1.0})
               })
run_experiment(config, project_name)

In [ ]:
config = dict({'model_label' : 'FCNN',
                'version' : 'patch_in3_out1_pres_loss_3D_norm_nf',
                'torch_model' : 'myParam.FCNN',
                'datamodule' : 'all_data_3D',
                'torch_model_params' : dict({'data_geometry' : '3D',\
                                            'nb_of_input_features' : 1, \
                                            'nb_of_output_features' : 1, \
                                            'input_patch_size' : 3,
                                            'output_patch_size' : 1, 
                                            'int_layer_width' : 50}),
                'module_params' : dict({'input_features'  : ['normalized_votemper'],
                                        'output_features'  : ['votemper_var'],
                                        'output_units' : ['diff_votemper_sqr'],
                                        'loss' : 'pressure_based_MSEloss',
                                        'optimizer' : torch.optim.Adam,
                                        'learning_rate' : 1e-3,}),
                'training_params' : dict({'max_epochs' : 100,
                                          'limit_train_batches' : 1.0})
               })
run_experiment(config, project_name)

In [ ]:
config = dict({'model_label' : 'FCNN',
                'version' : 'patch_in3_out1_pres_loss_3D_norm_fil',
                'torch_model' : 'myParam.FCNN',
                'datamodule' : 'all_data_3D',
                'torch_model_params' : dict({'data_geometry' : '3D',\
                                            'nb_of_input_features' : 1, \
                                            'nb_of_output_features' : 1, \
                                            'input_patch_size' : 3,
                                            'output_patch_size' : 1, 
                                            'int_layer_width' : 50}),
                'module_params' : dict({'input_features'  : ['normalized_votemper'],
                                        'output_features'  : ['votemper_var'],
                                        'output_units' : ['filtered_diff_votemper_sqr'],
                                        'loss' : 'pressure_based_MSEloss',
                                        'optimizer' : torch.optim.Adam,
                                        'learning_rate' : 1e-3,}),
                'training_params' : dict({'max_epochs' : 100,
                                          'limit_train_batches' : 1.0})
               })
run_experiment(config, project_name)

In [ ]:
config = dict({'model_label' : 'FCNN',
                'version' : 'patch_in3_out1_pres_loss_3D_nn_fil',
                'torch_model' : 'myParam.FCNN',
                'datamodule' : 'all_data_3D',
                'torch_model_params' : dict({'data_geometry' : '3D',\
                                            'nb_of_input_features' : 1, \
                                            'nb_of_output_features' : 1, \
                                            'input_patch_size' : 3,
                                            'output_patch_size' : 1, 
                                            'int_layer_width' : 50}),
                'module_params' : dict({'input_features'  : ['votemper'],
                                        'output_features'  : ['votemper_var'],
                                        'output_units' : ['filtered_diff_votemper_sqr'],
                                        'loss' : 'pressure_based_MSEloss',
                                        'optimizer' : torch.optim.Adam,
                                        'learning_rate' : 1e-3,}),
                'training_params' : dict({'max_epochs' : 100,
                                          'limit_train_batches' : 1.0})
               })
run_experiment(config, project_name)

In [ ]:
config = dict({'model_label' : 'FCNN',
                'version' : 'patch_in3_out1_pres_loss_3D_nn_dim',
                'torch_model' : 'myParam.FCNN',
                'datamodule' : 'all_data_3D',
                'torch_model_params' : dict({'data_geometry' : '3D',\
                                            'nb_of_input_features' : 1, \
                                            'nb_of_output_features' : 1, \
                                            'input_patch_size' : 3,
                                            'output_patch_size' : 1, 
                                            'int_layer_width' : 50}),
                'module_params' : dict({'input_features'  : ['votemper'],
                                        'output_features'  : ['votemper_var'],
                                        'output_units' : None,
                                        'loss' : 'pressure_based_MSEloss',
                                        'optimizer' : torch.optim.Adam,
                                        'learning_rate' : 1e-3,}),
                'training_params' : dict({'max_epochs' : 100,
                                          'limit_train_batches' : 1.0})
               })
run_experiment(config, project_name)

In [ ]:
config = dict({'model_label' : 'FCNN',
                'version' : 'patch_in3_out1_pres_loss_3D_norm_dim',
                'torch_model' : 'myParam.FCNN',
                'datamodule' : 'all_data_3D',
                'torch_model_params' : dict({'data_geometry' : '3D',\
                                            'nb_of_input_features' : 1, \
                                            'nb_of_output_features' : 1, \
                                            'input_patch_size' : 3,
                                            'output_patch_size' : 1, 
                                            'int_layer_width' : 50}),
                'module_params' : dict({'input_features'  : ['normalized_votemper'],
                                        'output_features'  : ['votemper_var'],
                                        'output_units' : None,
                                        'loss' : 'pressure_based_MSEloss',
                                        'optimizer' : torch.optim.Adam,
                                        'learning_rate' : 1e-3,}),
                'training_params' : dict({'max_epochs' : 100,
                                          'limit_train_batches' : 1.0})
               })
run_experiment(config, project_name)

## CNN

In [ ]:
config = dict({'model_label' : 'CNN',
                'version' : 'kernel3_val_3D_nn_nf',
                'torch_model' : 'myParam.CNN',
                'datamodule' : 'all_data_3D',
                'torch_model_params' : dict({'data_geometry' : '3D',\
                                            'nb_of_input_features' : 1, \
                                            'nb_of_output_features' : 1, \
                                             'kernel_size' : 3, \
                                            'int_layer_width' : 64}),
                'module_params' : dict({'input_features'  : ['votemper'],
                                        'output_features'  : ['votemper_var'],
                                        'output_units' : ['diff_votemper_sqr'],
                                        'loss' : torch.nn.functional.mse_loss,
                                        'optimizer' : torch.optim.Adam,
                                        'learning_rate' : 1e-3,}),
                'training_params' : dict({'max_epochs' : 100,
                                          'limit_train_batches' : 1.0})
               })

In [ ]:
config = dict({'model_label' : 'CNN',
                'version' : 'kernel3_hubval_3D_nn_nf',
                'torch_model' : 'myParam.CNN',
                'datamodule' : 'all_data_3D',
                'torch_model_params' : dict({'data_geometry' : '3D',\
                                            'nb_of_input_features' : 1, \
                                            'nb_of_output_features' : 1, \
                                             'kernel_size' : 3, \
                                            'int_layer_width' : 64}),
                'module_params' : dict({'input_features'  : ['votemper'],
                                        'output_features'  : ['votemper_var'],
                                        'output_units' : ['diff_votemper_sqr'],
                                        'loss' : torch.nn.functional.huber_loss,
                                        'optimizer' : torch.optim.Adam,
                                        'learning_rate' : 1e-3,}),
                'training_params' : dict({'max_epochs' : 100,
                                          'limit_train_batches' : 1.0})
               })

In [ ]:
config = dict({'model_label' : 'CNN',
                'version' : 'kernel3_val_3D_norm_nf',
                'torch_model' : 'myParam.CNN',
                'datamodule' : 'all_data_3D',
                'torch_model_params' : dict({'data_geometry' : '3D',\
                                            'nb_of_input_features' : 1, \
                                            'nb_of_output_features' : 1, \
                                             'kernel_size' : 3, \
                                            'int_layer_width' : 64}),
                'module_params' : dict({'input_features'  : ['normalized_votemper'],
                                        'output_features'  : ['votemper_var'],
                                        'output_units' : ['diff_votemper_sqr'],
                                        'loss' : torch.nn.functional.mse_loss,
                                        'optimizer' : torch.optim.Adam,
                                        'learning_rate' : 1e-3,}),
                'training_params' : dict({'max_epochs' : 100,
                                          'limit_train_batches' : 1.0})
               })

In [ ]:
config = dict({'model_label' : 'CNN',
                'version' : 'kernel3_hubval_3D_norm_nf',
                'torch_model' : 'myParam.CNN',
                'datamodule' : 'all_data_3D',
                'torch_model_params' : dict({'data_geometry' : '3D',\
                                            'nb_of_input_features' : 1, \
                                            'nb_of_output_features' : 1, \
                                             'kernel_size' : 3, \
                                            'int_layer_width' : 64}),
                'module_params' : dict({'input_features'  : ['normalized_votemper'],
                                        'output_features'  : ['votemper_var'],
                                        'output_units' : ['diff_votemper_sqr'],
                                        'loss' : torch.nn.functional.huber_loss,
                                        'optimizer' : torch.optim.Adam,
                                        'learning_rate' : 1e-3,}),
                'training_params' : dict({'max_epochs' : 100,
                                          'limit_train_batches' : 1.0})
               })